# Selector Group Chat

In [13]:
import asyncio
from typing import Sequence

from autogen_agentchat.agents import (
    BaseChatAgent,
    ChatMessage,
    CodingAssistantAgent,
    StopMessage,
    TextMessage,
    ToolUseAssistantAgent,
)
from autogen_agentchat.teams.group_chat import SelectorGroupChat
from autogen_core.base import CancellationToken
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

In [14]:
class UserProxyAgent(BaseChatAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name, "A human user.")

    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> ChatMessage:
        user_input = await asyncio.get_event_loop().run_in_executor(None, input, "Enter your response: ")
        if "TERMINATE" in user_input:
            return StopMessage(content="User has terminated the conversation.", source=self.name)
        return TextMessage(content=user_input, source=self.name)

In [15]:
async def flight_search(start: str, destination: str, date: str) -> str:
    return "\n".join(
        [
            f"AC24 from {start} to {destination} on {date} is $500",
            f"UA23 from {start} to {destination} on {date} is $450",
            f"AL21 from {start} to {destination} on {date} is $400",
        ]
    )


async def flight_booking(flight: str, date: str) -> str:
    return f"Booked flight {flight} on {date}"

In [16]:
user_proxy = UserProxyAgent("User")
flight_broker = ToolUseAssistantAgent(
    "FlightBroker",
    description="An assistant for booking flights",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    registered_tools=[
        FunctionTool(flight_search, description="Search for flights"),
        FunctionTool(flight_booking, description="Book a flight"),
    ],
)
travel_assistant = CodingAssistantAgent(
    "TravelAssistant",
    description="A travel assistant",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    system_message="You are a travel assistant.",
)
team = SelectorGroupChat(
    [user_proxy, flight_broker, travel_assistant], model_client=OpenAIChatCompletionClient(model="gpt-4o-mini")
)
await team.run("Help user plan a trip and book a flight.")


--------------------------------------------------------------------------- 
[2024-10-08T10:10:50.523469]:

Help user plan a trip and book a flight.
--------------------------------------------------------------------------- 
[2024-10-08T10:10:51.234858], group_chat_manager:

Selected next speaker: User
--------------------------------------------------------------------------- 
[2024-10-08T10:10:55.437051], User:


--------------------------------------------------------------------------- 
[2024-10-08T10:10:55.957366], group_chat_manager:

Selected next speaker: TravelAssistant
--------------------------------------------------------------------------- 
[2024-10-08T10:10:58.291558], TravelAssistant:

Sure! I can help you plan your trip and provide information on booking a flight. Could you please provide me with the following details?

1. Your departure city.
2. Your destination.
3. Travel dates (departure and return).
4. Number of travelers and their ages.
5. Any specific preferenc

TeamRunResult(messages=[TextMessage(source='user', content='Help user plan a trip and book a flight.'), TextMessage(source='User', content=''), TextMessage(source='TravelAssistant', content='Sure! I can help you plan your trip and provide information on booking a flight. Could you please provide me with the following details?\n\n1. Your departure city.\n2. Your destination.\n3. Travel dates (departure and return).\n4. Number of travelers and their ages.\n5. Any specific preferences or activities you would like to include in your trip?\n\nOnce I have that information, we can get started!'), TextMessage(source='User', content='Going to toronto from new york '), TextMessage(source='TravelAssistant', content='Great! Here are a few more details I need to help you plan your trip:\n\n1. **Departure dates:** When do you plan to leave New York and when will you return?\n2. **Number of travelers:** How many people will be traveling with you, and what are their ages?\n3. **Preferences:** Do you h